<a href="https://colab.research.google.com/github/jphilip86/claims-analysis/blob/main/notebooks/claims_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Part 1: Data Loading and Exploration

import pandas as pd

# Load the CSV files that were attached
# Reads three CSV files into pandas DataFrames:
# df_header: likely contains claim-level information.
# df_line: likely contains service line details for each claim.
# df_code: likely contains diagnosis codes per claim.


df_header = pd.read_csv('/content/drive/My Drive/Claims/STONYBRK_20240531_HEADER.csv')
df_line = pd.read_csv('/content/drive/My Drive/Claims/STONYBRK_20240531_LINE.csv')
df_code = pd.read_csv('/content/drive/My Drive/Claims/STONYBRK_20240531_CODE.csv')


In [16]:
# Function to display basic exploration info

def explore_df(df):
    return {
        'shape': df.shape,
        'head': df.head(),
        'columns_dtypes': df.dtypes,
        'missing_values': df.isnull().sum(),
        'describe': df.describe()
    }

In [17]:
# Explore each dataframe
data_header = explore_df(df_header)
data_line = explore_df(df_line)
data_code = explore_df(df_code)

(
    data_header['shape'], data_header['head'], data_header['columns_dtypes'], data_header['missing_values'], data_header['describe'],
    data_line['shape'], data_line['head'], data_line['columns_dtypes'], data_line['missing_values'], data_line['describe'],
    data_code['shape'], data_code['head'], data_code['columns_dtypes'], data_code['missing_values'], data_code['describe']
)

((388, 43),
    ClientId  ProspectiveClaimId ClaimType  BillingProviderNPI  \
 0        49            35216986         P          1154376309   
 1        49            35235657         P          1154376309   
 2        49            35249003         P          1154376309   
 3        49            35249014         P          1154376309   
 4        49            35249016         P          1154376309   
 
                BillingProvFirstName  BillingProvLastName  \
 0  NEW YORK SPINE AND BRAIN SURGERY                  NaN   
 1  NEW YORK SPINE AND BRAIN SURGERY                  NaN   
 2  NEW YORK SPINE AND BRAIN SURGERY                  NaN   
 3  NEW YORK SPINE AND BRAIN SURGERY                  NaN   
 4  NEW YORK SPINE AND BRAIN SURGERY                  NaN   
 
    AttendingProviderNPI AttendingProvFirstName AttendingProvLastName  \
 0                   NaN                    NaN                   NaN   
 1                   NaN                    NaN                   NaN   
 2 

In [18]:
# Additional exploration for observations

# How many unique claims?
num_unique_claims = df_header['ProspectiveClaimId'].nunique()

# What is the date range of the claims? This converts the 'ServiceFromDate' column
# from the header dataframe into datetime format. It then extracts the minimum and
# maximum dates to provide the date range of the claims,
# ensuring invalid dates are coerced to NaT (missing).

claim_dates = pd.to_datetime(df_header['ServiceFromDate'], errors='coerce')
min_date = claim_dates.min()
max_date = claim_dates.max()


# How many service lines are there on average per claim? (from df_line)
#This groups the service line data by claim IDs, counts the number of lines per claim, and calculates
# the average number of service lines per claim, offering insight into claim complexity.

avg_service_lines_per_claim = df_line.groupby('ProspectiveClaimId').size().mean()

# How many diagnosis codes are there on average per claim? (from df_code)
# Similarly, this calculates the average number of diagnosis codes per claim by
# grouping by claim ID in the diagnosis code dataframe and computing the mean count.
avg_diagnosis_codes_per_claim = df_code.groupby('ProspectiveClaimId').size().mean()

num_unique_claims, min_date, max_date, avg_service_lines_per_claim, avg_diagnosis_codes_per_claim

/tmp/ipython-input-999548616.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  claim_dates = pd.to_datetime(df_header['ServiceFromDate'], errors='coerce')


(388,
 Timestamp('2023-09-25 00:00:00'),
 Timestamp('2024-05-29 00:00:00'),
 np.float64(1.3402061855670102),
 np.float64(3.9587628865979383))

In [19]:
# Part 2: Relational Data Analysis
# analyze top providers, payers, diagnoses, procedures, and service locations

import matplotlib.pyplot as plt

# Question 1: Provider Analysis - Top 5 billing providers by number of claims

# Group by billing provider name and NPI, count unique claims
# Groups df_header by provider name and NPI (National Provider Identifier)
# For each provider, counts the unique number of ProspectiveClaimId (claims submitted by this provider).
# Renames the claims count column for clarity.
provider_claim_counts = df_header.groupby(['BillingProvFirstName', 'BillingProviderNPI'])['ProspectiveClaimId'].nunique().reset_index()
provider_claim_counts.rename(columns={'ProspectiveClaimId': 'ClaimCount'}, inplace=True)

# Sorts providers by number of claims, descending, and keeps the top 5.
provider_claim_counts_top5 = provider_claim_counts.sort_values(by='ClaimCount', ascending=False).head(5)

# Question 2: Payer Mix Analysis - Top 5 primary payers by claim volume
# Counts the occurrences of each payer in the PrimaryPayerName column,
# giving total claims for each payer.

# Renames columns to be clear.
payer_claim_counts = df_header['PrimaryPayerName'].value_counts().reset_index()
payer_claim_counts.columns = ['PrimaryPayerName', 'ClaimCount']

# Calculates the percentage each payer makes up of the total claims.Keeps the top 5 payers.
payer_claim_counts['ClaimPercent'] = 100 * payer_claim_counts['ClaimCount'] / payer_claim_counts['ClaimCount'].sum()
# Top 5
payer_claim_counts_top5 = payer_claim_counts.head(5)

# Question 3: Common Diagnoses - 10 most frequent diagnosis codes
# Counts frequency of each diagnosis code (CodeValue) in df_code.
# Makes a summary table for the 10 most common ICD-10 diagnosis codes.

common_diagnoses = df_code['CodeValue'].value_counts().reset_index()
common_diagnoses.columns = ['ICD10Code', 'Frequency']
common_diagnoses_top10 = common_diagnoses.head(10)

# Question 4: Common Procedures - 10 most frequent HCPCS codes
# Counts frequency of each HCPCS code in the service line data.Keeps the top 10 procedures.
common_procedures = df_line['HCPCS'].value_counts().reset_index()
common_procedures.columns = ['HCPCSCode', 'Frequency']
common_procedures_top10 = common_procedures.head(10)

# Joins the top procedures with their readable descriptions (ClientProcedureName),
# dropping duplicates for clarity.
procedures_with_desc = common_procedures_top10.merge(df_line[['HCPCS', 'ClientProcedureName']].drop_duplicates(), left_on='HCPCSCode', right_on='HCPCS', how='left').drop(columns=['HCPCS'])

# Question 5: Service Location Analysis
# How many claims submitted for each PlaceOfService?
# Counts the frequency of each PlaceOfService code in all claims.

place_of_service_counts = df_header['PlaceOfService'].value_counts().reset_index()
place_of_service_counts.columns = ['PlaceOfService', 'ClaimCount']

# Map PlaceOfService codes to descriptive names for clarity - from df_header if available
# The dataset does not provide explicit mapping, but from observation '21' is common for inpatient and typical outpatient codes like 11 for doctor's office
place_of_service_map = {
    21: 'INPATIENT',
    11: 'DOCTOR\'S OFFICE'
}
place_of_service_counts['PlaceOfServiceDesc'] = place_of_service_counts['PlaceOfService'].map(place_of_service_map)

# Calculate % of claims for INPATIENT vs DOCTOR'S OFFICE
total_claims = place_of_service_counts['ClaimCount'].sum()
percent_inpatient = place_of_service_counts[place_of_service_counts['PlaceOfServiceDesc'] == 'INPATIENT']['ClaimCount'].sum() / total_claims * 100
percent_doctors_office = place_of_service_counts[place_of_service_counts['PlaceOfServiceDesc'] == "DOCTOR'S OFFICE"]['ClaimCount'].sum() / total_claims * 100

(provider_claim_counts_top5, payer_claim_counts_top5, common_diagnoses_top10, procedures_with_desc, place_of_service_counts, percent_inpatient, percent_doctors_office)

(                  BillingProvFirstName  BillingProviderNPI  ClaimCount
 3                        SB INTERNISTS          1821035601         152
 5               SB SURGICAL ASSOCIATES          1063468536          81
 0     NEW YORK SPINE AND BRAIN SURGERY          1154376309          69
 6  UNIV.ASSOC.IN OBSTETRICS&GYNECOLOGY          1538114723          40
 4            SB PSYCHIATRIC ASSOCIATES          1437105905          36,
              PrimaryPayerName  ClaimCount  ClaimPercent
 0                    MEDICARE         242     62.371134
 1             HEALTHFIRST FFS          46     11.855670
 2  FIDELIS/BETTER HEALTH PLAN          26      6.701031
 3                HIP MEDICAID          17      4.381443
 4       HEALTHFIRST CAPITATED          10      2.577320,
   ICD10Code  Frequency
 0    J96.01         62
 1       I10         49
 2     E78.5         49
 3     G93.5         34
 4     D64.9         29
 5    I25.10         27
 6     I61.9         26
 7    I48.91         24
 8     I

In [20]:
# Top 5 Billing Providers by Number of Claims - Bar Chart

import plotly.graph_objects as go
import plotly.colors as pc

# Data provided
provider_names = ["SB INTERNISTS", "SB SURGICAL ASSOCIATES", "NEW YORK SPINE AND BRAIN SURGERY", "UNIV.ASSOC.IN OBSTETRICS&GYNECOLOGY", "SB PSYCHIATRIC ASSOCIATES"]
claim_counts = [152, 81, 69, 40, 36]

# Abbreviated provider names for display
abbreviated_names = [
    "SB INTERNISTS",
    "SB SURGICAL",
    "NY SPINE&BRAIN",
    "UNIV OB&GYN",
    "SB PSYCHIATRIC"
]

# Choose a palette for 5 bars
colors = pc.qualitative.Plotly[:5]  # Just the first 5 colors

# Create improved bar chart
fig = go.Figure(data=[
    go.Bar(
        x=abbreviated_names,
        y=claim_counts,
        marker_color=colors,
        marker_line_width=2,
        marker_line_color="white",
        hovertemplate='<b>%{x}</b><br>Claims: %{y}<extra></extra>'
    )
])

fig.update_layout(
    title="Top 5 Providers by Claims",
    title_font_size=24,
    xaxis_title="Provider Name",
    xaxis_title_font_size=18,
    yaxis_title="Claim Count",
    font=dict(size=15, family="Arial"),
    plot_bgcolor="#FAFBFD",
    paper_bgcolor="#F6F8FA",
    height=500,
    margin=dict(l=70, r=50, t=80, b=40)
)

fig.update_yaxes(showgrid=True, gridcolor="#E5ECF6", zeroline=False)
fig.update_xaxes(showline=False, showgrid=False)
fig.update_traces(cliponaxis=False)

fig.add_annotation(
    text="Source: Claims Data Analysis",
    xref="paper", yref="paper",
    x=0, y=-0.15, showarrow=False,
    font=dict(size=12, color="gray")
)

fig.show()



In [21]:
# Claim Volume Distribution by Top 5 Primary Payers - Pie Chart

import plotly.graph_objects as go

# Data
payers = ["MEDICARE", "HFIRST FFS", "FIDELIS/BETTER", "HIP MEDICAID", "HFIRST CAP"]
claims = [242, 46, 26, 17, 10]

# Brand colors for pie chart
colors = ['#1FB8CD', '#DB4545', '#2E8B57', '#5D878F', '#D2BA4C']

# Create pie chart
fig = go.Figure(data=[go.Pie(
    labels=payers,
    values=claims,
    textposition='inside',
    textinfo='label+percent',
    marker=dict(colors=colors)
)])

# Update layout
fig.update_layout(
    title="Top 5 Payer Claims",
    uniformtext_minsize=14,
    uniformtext_mode='hide'
)


In [22]:
# Top 10 Most Frequently Billed HCPCS Procedure Codes

import plotly.graph_objects as go
import json

# Load the data
data = {
    "hcpcs_codes": ["99291", "99233", "99213", "99223", "99222", "99232", "90833", "99214", "92557", "99204"],
    "frequencies": [68, 48, 39, 33, 32, 21, 16, 14, 14, 14],
    "descriptions_short": ["Crit Care 1hr", "Subs Hosp Care", "Outpt Visit Est", "Init Hosp Care", "Init Hosp Care", "Subs Hosp Care", "Psychotherapy", "Outpt Visit Est", "Audiometry", "Outpt Visit New"],
    "descriptions_full": ["CRITICAL CARE, INITIAL FIRST HOUR", "SUBSEQUENT HOSP. CARE,PER DAY,FOR THE E&M OF A...", "OFFICE/OUTPATIENT VISIT,ESTABL.PT,EXPANDED HX,...", "INITIAL  HOSPITAL CARE PER DAY,FOR THE E&M OF ...", "INITIAL HOSP. CARE PER DAY FOR THE E&M OF ...", "SUBS.HOSP.CARE EXPAND.PROBLEM FOCUSED HX.,EXAM...", "PSYCHOTHERAPY WITH AN E&M SERVICE,30 MINS,PATI...", "OFFICE/OUTPT VISIT,ESTAB. PATIENT,DETAILED HX,...", "COMPREHENSIVE AUDIOMETRY THRESHOLD EVALUATION;...", "OFFICE/OUTPT VISIT FOR  THE E & M OF A NEW PAT..."]
}

# Create combined labels for y-axis
y_labels = [f"{code} - {desc}" for code, desc in zip(data['hcpcs_codes'], data['descriptions_short'])]

# Define colors - using the 10 brand colors provided
colors = ['#1FB8CD', '#DB4545', '#2E8B57', '#5D878F', '#D2BA4C',
          '#B4413C', '#964325', '#944454', '#13343B', '#DB4545']

# Create the horizontal bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=data['frequencies'],
    y=y_labels,
    orientation='h',
    marker=dict(color=colors),
    customdata=list(zip(data['descriptions_full'], data['frequencies'])),
    hovertemplate='<b>%{customdata[0]}</b><br>Frequency: %{customdata[1]}<extra></extra>'
))

# Update layout
fig.update_layout(
    title='Top 10 HCPCS Codes',
    xaxis_title='Frequency',
    yaxis_title='',
    yaxis={'categoryorder': 'total ascending'}  # This will put highest frequency at top
)

fig.update_traces(cliponaxis=False)

In [23]:
# Part 3: Advanced Analysis with Joins
# Question 6: Claims with High Service Line Counts

# Merge HEADER and LINE on ProspectiveClaimId
header_line_merged = pd.merge(df_header, df_line, on='ProspectiveClaimId', suffixes=('_header', '_line'))

# Calculate total number of service lines per claim & total charges per claim
service_lines_count = df_line.groupby('ProspectiveClaimId').size().reset_index(name='NumServiceLines')
service_lines_charges = df_line.groupby('ProspectiveClaimId')['Charges'].sum().reset_index(name='TotalCharges')

# Merge counts and charges with HEADER to display Provider name
high_service_lines = pd.merge(df_header[['ProspectiveClaimId', 'BillingProvFirstName']], service_lines_count, on='ProspectiveClaimId')
high_service_lines = pd.merge(high_service_lines, service_lines_charges, on='ProspectiveClaimId')

# Filter claims with 5 or more service lines
claims_5plus_lines = high_service_lines[high_service_lines['NumServiceLines'] >= 5][['ProspectiveClaimId', 'BillingProvFirstName', 'NumServiceLines', 'TotalCharges']].reset_index(drop=True)

# Question 7: Diagnosis-Procedure Combinations
# Merge all three datasets: HEADER, LINE, CODE
merged_header_code = pd.merge(df_header, df_code, on='ProspectiveClaimId', suffixes=('_header', '_code'))
merged_full = pd.merge(merged_header_code, df_line, on='ProspectiveClaimId', suffixes=('_header_code', '_line'))

# Filter for CPT code 99291 in LINE data
cpt_99291 = merged_full[merged_full['HCPCS'] == '99291']

# Find the most common diagnosis code (CodeValue) associated with CPT 99291
common_diag_for_99291 = cpt_99291['CodeValue'].value_counts().reset_index().rename(columns={'index': 'CodeValue', 'CodeValue': 'Frequency'}).head(1)

# Question 8: Charges by Payer
# Total charges per claim
total_charges_claim = df_line.groupby('ProspectiveClaimId')['Charges'].sum().reset_index()

# Merge total charges with HEADER for payer info
charges_payer = pd.merge(df_header[['ProspectiveClaimId', 'PrimaryPayerName']], total_charges_claim, on='ProspectiveClaimId')

# Group by PrimaryPayerName and calculate total charges, avg charge per claim and claim count
payer_charges_summary = charges_payer.groupby('PrimaryPayerName').agg(
    TotalCharges=pd.NamedAgg(column='Charges', aggfunc='sum'),
    AvgChargePerClaim=pd.NamedAgg(column='Charges', aggfunc='mean'),
    NumClaims=pd.NamedAgg(column='Charges', aggfunc='count')
).reset_index()

payer_charges_summary_sorted = payer_charges_summary.sort_values('TotalCharges', ascending=False).head(10)

(claims_5plus_lines, common_diag_for_99291, payer_charges_summary_sorted)

(   ProspectiveClaimId                 BillingProvFirstName  NumServiceLines  \
 0            36710175  UNIV.ASSOC.IN OBSTETRICS&GYNECOLOGY                5   
 1            36740402  UNIV.ASSOC.IN OBSTETRICS&GYNECOLOGY                6   
 2            36668119  UNIV.ASSOC.IN OBSTETRICS&GYNECOLOGY                6   
 3            36757684  UNIV.ASSOC.IN OBSTETRICS&GYNECOLOGY                5   
 4            36794825                SB CHILDREN'S SERVICE                7   
 
    TotalCharges  
 0           873  
 1           945  
 2          1030  
 3           873  
 4          1163  ,
   Frequency  count
 0    J96.01     53,
                       PrimaryPayerName  TotalCharges  AvgChargePerClaim  \
 17                            MEDICARE        131008         541.355372   
 12                     HEALTHFIRST FFS         29794         647.695652   
 9           FIDELIS/BETTER HEALTH PLAN         10810         415.769231   
 14                        HIP MEDICAID         10014     

In [24]:
# Part 4: Creative Analysis

# Which billing providers submit claims with the greatest diversity of diagnosis codes per claim?
# This reveals who handles the most clinically complex or multi-condition cases.
# Providers who see more complicated or multi-condition patients will tend to bill claims listing more diagnosis codes.
# Measuring this can reflect case complexity, resource needs, or catch billing anomalies.

In [25]:
# STEP 1
# Merge header and code by ProspectiveClaimId to associate each claim with provider and diagnosis codes
provider_code_merged = pd.merge(
    df_header[['ProspectiveClaimId', 'BillingProvFirstName', 'BillingProviderNPI']],
    df_code[['ProspectiveClaimId', 'CodeValue']],
    on='ProspectiveClaimId'
)
# Joins the claim-level info (including which provider submitted which claim) to the diagnosis code records for each claim, linking each provider to every diagnosis code on every claim they’ve billed.
# The merged DataFrame now has one row per diagnosis code per claim,
# with clear attribution to provider and claim.

In [26]:
# STEP 2
# Count distinct diagnosis codes per claim
diag_codes_per_claim = provider_code_merged.groupby(
    ['BillingProviderNPI', 'BillingProvFirstName', 'ProspectiveClaimId']
)['CodeValue'].nunique().reset_index(name='NumDiagnosisCodes')

# STEP 3
# Now average across claims for each provider
provider_diag_complexity = diag_codes_per_claim.groupby(
    ['BillingProviderNPI', 'BillingProvFirstName']
)['NumDiagnosisCodes'].mean().reset_index(name='AvgCodesPerClaim')
# Some providers might see mostly simple or mostly complex patients;
# averaging shows whose practice is the most clinically diverse or complex overall.

# Sort descending for top "most complex" providers
provider_diag_complexity_sorted = provider_diag_complexity.sort_values('AvgCodesPerClaim', ascending=False).head(10)


In [27]:
# STEP 4 VISUALIZATION
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure(data=[
    go.Bar(
        y=provider_diag_complexity_sorted['BillingProvFirstName'] + " (" + provider_diag_complexity_sorted['BillingProviderNPI'].astype(str) + ")",
        x=provider_diag_complexity_sorted['AvgCodesPerClaim'],
        orientation='h',
        marker_color=px.colors.qualitative.Plotly  # Each bar gets a color from the sequence
    )
])

fig.update_layout(
    title="Top 7 Providers by Average Diagnosis Codes Per Claim",
    xaxis_title="Average Number of Diagnosis Codes per Claim",
    yaxis_title="Provider (NPI)",
    yaxis={'categoryorder':'total ascending'}
)

fig.show()



In [28]:
# Conclusion/Insight:
# Providers who bill claims with many distinct diagnosis codes
# per claim are likely treating more complex patients—individuals with multiple comorbid conditions,
# chronic illnesses, or acute health needs.

# This can reflect provider specialization
# (e.g., hospitalists, intensivists, or specialists managing medically complicated cases),
# differences in patient populations, or areas of practice with high diagnostic uncertainty.

# These providers may require more resources,
# coordination, and multidisciplinary care.

# Such claims may merit closer clinical review
# (for quality assurance or auditing), as complex claims can be prone to procedural,
# documentation, or coding errors.

